In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import numpy as np

from selenium.webdriver.chrome.service import Service


def get_driver(use_proxy=False):

    chrome_options = Options()
    chrome_options = webdriver.ChromeOptions()

    if use_proxy:
        driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
        driver.get("https://sslproxies.org/")
        driver.execute_script("return arguments[0].scrollIntoView(true);", WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//table[@class='table table-striped table-bordered dataTable']//th[contains(., 'IP Address')]"))))
        ips = [my_elem.get_attribute("innerHTML") for my_elem in WebDriverWait(driver, 5).until(EC.visibility_of_all_elements_located((By.XPATH, "//table[@class='table table-striped table-bordered dataTable']//tbody//tr[@role='row']/td[position() = 1]")))]
        ports = [my_elem.get_attribute("innerHTML") for my_elem in WebDriverWait(driver, 5).until(EC.visibility_of_all_elements_located((By.XPATH, "//table[@class='table table-striped table-bordered dataTable']//tbody//tr[@role='row']/td[position() = 2]")))]
        proxies = []
        for i in range(0, len(ips)):
            proxies.append(ips[i]+':'+ports[i])
        chrome_options = webdriver.ChromeOptions()
        i = int(np.random.randint(low=0, high=5, size=(1,)))
        chrome_options.add_argument('--proxy-server={}'.format(proxies[i]))
        chrome_options.add_argument("start-maximized")
        chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
        chrome_options.add_experimental_option('useAutomationExtension', False)

    chrome_options.add_argument("start-maximized")
    service = Service()
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source":
        "const newProto = navigator.__proto__;"
        "delete newProto.webdriver;"
        "navigator.__proto__ = newProto;"
    })

    return driver

In [48]:
driver = get_driver()

base_url = ["https://paperswithcode.com/"]

In [6]:
driver.get(base_url)


In [7]:
html = driver.page_source
soup = BeautifulSoup(html)

In [15]:
soup.find_all("div", class_ = "col-lg-9 item-content")[0].find_all("a")[0].get("href")


'/paper/llama-omni-seamless-speech-interaction-with'

In [28]:
soup.find_all("div", class_ = "col-lg-9 item-content")[0].find_all("a")[0].text


'LLaMA-Omni: Seamless Speech Interaction with Large Language Models'

In [26]:
soup.find_all("div", class_ = "col-lg-9 item-content")[0].find_all("p", class_="item-strip-abstract")[0].text




'We build our model based on the latest Llama-3. 1-8B-Instruct model.'

In [30]:
bookmarks = []
for element in soup.find_all("div", class_ = "col-lg-9 item-content"):
    print(element.find_all("a")[0].get("href"))
    print(element.find_all("a")[0].text)
    print(element.find_all("p", class_="item-strip-abstract")[0].text)
    bookmarks.append(
        {
            "url": element.find_all("a")[0].get("href"),
            "title": element.find_all("a")[0].text,
            "description": element.find_all("p", class_="item-strip-abstract")[0].text,
        }
    )

/paper/llama-omni-seamless-speech-interaction-with
LLaMA-Omni: Seamless Speech Interaction with Large Language Models
We build our model based on the latest Llama-3. 1-8B-Instruct model.
/paper/pulid-pure-and-lightning-id-customization-via
PuLID: Pure and Lightning ID Customization via Contrastive Alignment
We propose Pure and Lightning ID customization (PuLID), a novel tuning-free ID customization method for text-to-image generation.
/paper/paperqa-retrieval-augmented-generative-agent
PaperQA: Retrieval-Augmented Generative Agent for Scientific Research
We present PaperQA, a RAG agent for answering questions over the scientific literature.
/paper/windows-agent-arena-evaluating-multi-modal-os
Windows Agent Arena: Evaluating Multi-Modal OS Agents at Scale
To demonstrate Windows Agent Arena's capabilities, we also introduce a new multi-modal agent, Navi.
https://github.com/codelion/optillm
optillm
Optimizing inference proxy for LLMs
/paper/mini-omni-language-models-can-hear-talk-while
Mi

In [31]:
import polars as pl

pl.DataFrame(bookmarks)


url,title,description
str,str,str
"""/paper/llama-omni-seamless-spe…","""LLaMA-Omni: Seamless Speech In…","""We build our model based on th…"
"""/paper/pulid-pure-and-lightnin…","""PuLID: Pure and Lightning ID C…","""We propose Pure and Lightning …"
"""/paper/paperqa-retrieval-augme…","""PaperQA: Retrieval-Augmented G…","""We present PaperQA, a RAG agen…"
"""/paper/windows-agent-arena-eva…","""Windows Agent Arena: Evaluatin…","""To demonstrate Windows Agent A…"
"""https://github.com/codelion/op…","""optillm""","""Optimizing inference proxy for…"
"""/paper/mini-omni-language-mode…","""Mini-Omni: Language Models Can…","""We also introduce the VoiceAss…"
"""/paper/agent-workflow-memory""","""Agent Workflow Memory""","""Despite the potential of langu…"
"""/paper/hi3d-pursuing-high-reso…","""Hi3D: Pursuing High-Resolution…","""Despite having tremendous prog…"
"""/paper/do-anything-now-charact…","""""Do Anything Now"": Characteriz…","""We hope that our study can fac…"


## Create a component

In [ ]:
from pydantic import BaseModel

class WebpageIntegration(BaseModel):
    base_urls: list[str]
    driver: webdriver.Chrome = None
    soup: BeautifulSoup = None

    model_config = {
        "arbitrary_types_allowed": True
    }

    def __init__(self, **data):
        super().__init__(**data)
        if self.driver is None:
            self.driver = get_driver()

    def load_page(self, url: str):
        self.driver.get(url)
        html = self.driver.page_source
        self.soup = BeautifulSoup(html, 'html.parser')

    def get_element_by_selector(self, selector: str, attr: str = None):
        element = self.soup.select_one(selector)
        if element:
            return element.get(attr) if attr else element.text.strip()
        return None

    def get_elements_by_selector(self, selector: str, attr: str = None):
        elements = self.soup.select(selector)
        if attr:
            return [element.get(attr) for element in elements if element.get(attr)]
        return [element.text.strip() for element in elements]

    def get_body(self):
        return self.get_element_by_selector('body')
    
    def _compose_bookmark(self, element):
        return {
            "url": element.find_all("a")[0].get("href"),
            "title": element.find_all("a")[0].text,
            "description": element.find_all("p", class_="item-strip-abstract")[0].text,
        }
    
    def get_bookmarks(self):
        bookmarks = []
        for base_url in self.base_urls:
            self.load_page(base_url)
            for element in self.soup.find_all("div", class_ = "col-lg-9 item-content"):
                bookmarks.append(self._compose_bookmark(element))
        return bookmarks
    
    def close(self):
        if self.driver:
            self.driver.quit()


In [ ]:
wbi = WebpageIntegration(base_urls=base_url, driver=driver)


In [ ]:
l_bookmarks = wbi.get_bookmarks()

pl.DataFrame(l_bookmarks)
